# Google Colab RLセットアップ
Google Colab上で麻雀強化学習環境をセットアップし、学習を実行します。

# # 麻雀 Transformer AI 強化学習 (PPO + Custom Policy)

**目的:** 事前学習済みの Transformer モデルを基盤として、Stable Baselines3 の PPO アルゴリズムを用い、カスタム麻雀環境 (`MahjongEnv`) で自己対戦学習を実行します。

---

## 1. 環境準備とライブラリインストール

### **GPU の選択:**
メニューから **「ランタイム」 > 「ランタイムのタイプを変更」** を選択し、**「ハードウェアアクセラレータ」** で **「GPU」** を選択してください。

In [ ]:
# 必要なライブラリのインストール
!pip install mahjong gymnasium stable-baselines3[extra] torch numpy
print("Required libraries installed.")

# --- GPU の利用状況を確認 ---
import torch
if torch.cuda.is_available():
    device = "cuda"
    print(f"✅ GPU available: {torch.cuda.get_device_name(0)}")
else:
    device = "cpu"
    print("❌ GPU not available. Using CPU.")

In [ ]:
from google.colab import drive
import os
import sys

# Google Driveをマウント
drive.mount('/content/gdrive')

# 【重要】プロジェクトのルートディレクトリをあなたの環境に合わせて設定してください
PROJECT_ROOT = '/content/gdrive/MyDrive/mahjong-rl-transformer'

# 作業ディレクトリを設定し、パスに追加
if not os.path.exists(PROJECT_ROOT):
    os.makedirs(PROJECT_ROOT)
os.chdir(PROJECT_ROOT)

if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

print(f"Working directory set to: {os.getcwd()}")

In [ ]:
# mahjong_rl_env/environment.py などからクラスをインポート
from mahjong_rl_env.environment import MahjongEnv
from mahjong_rl_env.feature_converter import get_context_vector 
from models.custom_policy import MaskedTransformerPolicy, TransformerFeatureExtractor
from stable_baselines3 import PPO

print("Custom Python modules loaded successfully.")

In [ ]:
# --- ハイパーパラメータ設定 ---
RL_AGENT_ID = 0           
FEATURES_DIM = 512        
LEARNING_RATE = 3e-5      
TOTAL_TIMESTEPS = 5_000_000 # 総学習ステップ数 (必要に応じて調整)
N_STEPS = 2048            

# 1. 環境のインスタンス化
env = MahjongEnv(agent_id=RL_AGENT_ID, opponent_policy=None) 
print(f"MahjongEnv created. RL Agent is Player {RL_AGENT_ID}.")


# 2. カスタムポリシーのための引数設定
policy_kwargs = dict(
    features_extractor_class=TransformerFeatureExtractor,
    features_extractor_kwargs=dict(
        features_dim=FEATURES_DIM,
        # ★ ここで TransformerFeatureExtractor 内の重みロードが実行されます
    ),
    # Actor/Critic の MLP 層の定義
    net_arch=[dict(pi=[256, 256], vf=[256, 256])] 
)

# 3. PPO モデルの初期化
MODEL_LOG_PATH = os.path.join(PROJECT_ROOT, "tensorboard_logs/mahjong_rl/")

model = PPO(
    MaskedTransformerPolicy,
    env,
    learning_rate=LEARNING_RATE,
    n_steps=N_STEPS,
    batch_size=N_STEPS, 
    n_epochs=10,
    gamma=0.99,
    verbose=1,
    tensorboard_log=MODEL_LOG_PATH,
    device=device,
    policy_kwargs=policy_kwargs
)

print("PPO Model and MaskedTransformerPolicy initialized.")
print("✅ Pre-trained weights should be loaded inside TransformerFeatureExtractor's __init__.")